In [7]:
using ProgressBars
using ITensors
using PastaQ
using RandomMeas

┌ Warning: /home/benoit/.julia/packages/Observers/1ausc/src/dataframe/observer.jl no longer exists, deleted all methods
└ @ Revise ~/.julia/packages/Revise/FaTes/src/packagedef.jl:666
┌ Warning: /home/benoit/.julia/packages/Observers/1ausc/src/base/method_utils.jl no longer exists, deleted all methods
└ @ Revise ~/.julia/packages/Revise/FaTes/src/packagedef.jl:666
┌ Warning: /home/benoit/.julia/packages/Observers/1ausc/src/dataframe/deprecated.jl no longer exists, deleted all methods
└ @ Revise ~/.julia/packages/Revise/FaTes/src/packagedef.jl:666
┌ Warning: /home/benoit/.julia/packages/Observers/1ausc/src/abstractdataframe/column_functions.jl no longer exists, deleted all methods
└ @ Revise ~/.julia/packages/Revise/FaTes/src/packagedef.jl:666
┌ Warning: /home/benoit/.julia/packages/Observers/1ausc/src/abstractdataframe/deprecated.jl no longer exists, deleted all methods
└ @ Revise ~/.julia/packages/Revise/FaTes/src/packagedef.jl:666
┌ Warning: /home/benoit/.julia/packages/ITensors/4M2e

In [8]:
N = 2
depth = 2
circuit    = randomcircuit(N, depth=depth)
noisemodel = (1 => ("depolarizing", (p = 0.02,)),2 => ("depolarizing", (p = 0.02,)))
noisemodel0 = (1 => ("depolarizing", (p = 0.0,)),2 => ("depolarizing", (p = 0.0,)))
Λ0 = runcircuit(circuit; process = true, noise = noisemodel0)
ξ0 = [firstind(Λ0[i],tags="Input",plev=0) for i in 1:N]
ξ20 = [firstind(Λ0[i],tags="Output",plev=0) for i in 1:N]
Λ = runcircuit(circuit; process = true, noise = noisemodel)
ξ = [firstind(Λ[i],tags="Input",plev=0) for i in 1:N]
ξ2 = [firstind(Λ[i],tags="Output",plev=0) for i in 1:N]
for i in 1:N
    Λ0[i] = replaceind(Λ0[i],ξ0[i],ξ[i])
    Λ0[i] = replaceind(Λ0[i],ξ0'[i],ξ'[i])
    Λ0[i] = replaceind(Λ0[i],ξ20[i],ξ2[i])
    Λ0[i] = replaceind(Λ0[i],ξ20'[i],ξ2'[i])
end
F = fidelity(Λ,Λ0)
println(" -- Process Fidelity ", F)

LoadError: MethodError: no method matching OpName(::String)

In [9]:
nu = 4000
NM = 10000
states = ["Dn" for n in 1:N]
ψ  = MPS(ComplexF64,ξ,states)
ρ = outer(ψ',ψ)
data = zeros(Int8,(NM,N))
shadow = ITensor(vcat(ξ,ξ'))
Λs = ITensor(vcat(ξ,ξ',ξ2,ξ2'))
Λs_CRM = ITensor(vcat(ξ,ξ',ξ2,ξ2'))

P0 = state(ξ[1],"Dn")
for i in 2:N
    P0 *= state(ξ[i],"Dn")
end

Fs = zeros(Float64,nu)
Fs_CRM = zeros(Float64,nu)

for r in ProgressBar(1:nu, printing_delay=2)   
        ui = get_rotations(ξ,1) #Haar rotations in A
        uf = get_rotations(ξ,1) #Haar rotations in A
        ### Quantum Measurements
        ρu = apply(ui,ρ,apply_dag=true)
        ρe = runcircuit(ρu,circuit,noise=noisemodel)
        #Build shadow for the initial state
        uid = [dag(swapinds(ui[i],ξ[i],ξ[i]')) for i in 1:N]
        ρi = get_shadow(P0,ξ,uid)
        ρi = swapinds(ρi,ξ,ξ')

        #Build shadow for the final state
        data = get_RandomMeas(ρe,uf,NM)
        P = get_Born(data,ξ)
        ρf = get_shadow(P,ξ,uf)
        replaceinds!(ρf,ξ,ξ2)
        replaceinds!(ρf,ξ',ξ2')

        #Form process shadow
        Λt = 2^N*ρi*ρf
        Λs += Λt
        Fs[r] = real(inner(Λs,flatten(Λ0))/2^(2*N))/r
    
        #Repeat the same thing for simulated measurements
        σe = runcircuit(ρu,circuit,noise=noisemodel0)
        Pσ = get_Born(apply(uf,σe,apply_dag=true))
        σf = get_shadow(Pσ,ξ,uf)
        replaceinds!(σf,ξ,ξ2)
        replaceinds!(σf,ξ',ξ2')
        Λt_CRM = Λt-2^N*ρi*σf+flatten(Λ0)
        Λs_CRM += Λt_CRM
        Fs_CRM[r] = real(inner(Λs_CRM,flatten(Λ0))/2^(2*N))/r
end

LoadError: MethodError: no method matching MPS(::Type{ComplexF64}, ::Vector{Index{Int64}}, ::Vector{String})

In [ ]:
using Plots
plot(1:nu,Fs,label="estimated")
plot!(1:nu,Fs_CRM,label="CRM estimated")
plot!(1:nu,F*ones(nu),"--k",label="exact")
xscale!("log")
yscale!("log")
ylim(ymax=1)
ylabel!("Fidelity")
xlabel!("unitaries")

[ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
[ Info: Precompiling FileIOExt [f5f51d8f-5827-5d2e-939b-192fcd6ec70c]
